In [1]:
import warnings
import os
from pathlib import Path

import pandas as pd
from google.cloud.bigquery.client import Client as BigQueryClient

from utils import *

warnings.filterwarnings("ignore", category=UserWarning)

big_query_client = BigQueryClient()

# MOD train

In [2]:
mod_train_version = "0.7.0"

query = f"""
    SELECT
        *
    FROM
        `noimos-core-axach-development.mod.train` AS train
"""

query_job = big_query_client.query(query)
mod_train_table = query_job.to_dataframe()

mod_train_rids = mod_train_table[mod_train_table["version"] == mod_train_version]["informationRequestId"].unique()

# MOD test

In [3]:
mod_test_version = "0.7.0"

query = f"""
    SELECT
        *
    FROM
        `noimos-core-axach-development.mod.test` AS train
"""

query_job = big_query_client.query(query)
mod_test_table = query_job.to_dataframe()

mod_test_rids = mod_test_table[mod_test_table["version"] == mod_test_version]["informationRequestId"].unique()

# Damage train / test

In [4]:
bucket_name = "noimos-data-lake-dataloop"


## Get all images in that bucket 
folder_name = "damage_workflow_batch_7"

storage_client = storage.Client()

# Get the bucket
bucket = storage_client.bucket(bucket_name)

# List the blobs in the folder
blobs = bucket.list_blobs(prefix=folder_name + '/')

file_paths = [blob.name for blob in blobs]

jpeg_files = filter_jpeg_files_direct_children(file_paths, folder_name)

image_names = [os.path.basename(file) for file in jpeg_files]




In [5]:
## Get the information request ids corresponding to those images
bq_client = BigQueryClient()

query = f"""
        SELECT DISTINCT
            app_case.id AS case_id,
            attachment.informationRequestId AS request_id,
            attachment.filename AS image_file_name
        FROM `noimos-data-lake-exp.prod.int_app_attachment` AS attachment
        INNER JOIN `noimos-data-lake-exp.prod.int_app_case` AS app_case
        ON attachment.informationrequestid = app_case.defaultinformationrequestid 
        WHERE NOT ENDS_WITH(attachment.filename, ".pdf") AND NOT ENDS_WITH(attachment.filename, ".xml");
"""

query_job = bq_client.query(query=query, project="noimos-data-lake-exp")
rows = list(query_job.result())
image_to_case = pd.DataFrame([dict(row) for row in rows])

In [8]:
Path(image_names[0])

PosixPath('-2pO-ccaFYS6stJsODXIy.jpeg')

In [6]:
image_to_case[image_to_case["image_file_name"].str.contains(rf'{Path(image_names[0]).stem}\.[a-zA-Z0-9]+$', regex=True) ]

,case_id,request_id,image_file_name


# Parts train/test

NameError: name 'np' is not defined